<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#증가율이-1이-될때까지-돌림" data-toc-modified-id="증가율이-1이-될때까지-돌림-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>증가율이 1이 될때까지 돌림</a></span></li><li><span><a href="#프라타-과정" data-toc-modified-id="프라타-과정-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>프라타 과정</a></span></li></ul></div>

In [12]:
import pandas as pd

df = pd.read_excel('./간단한 매트릭스.xlsx', index_col=0)

wntO = [28, 20, 12, 20]
wntD = [23, 20, 18, 19]

# df_copy = df.copy()

df

,0,1,2,3,4
0,7,6,5,8,26.0
1,5,4,6,2,17.0
2,2,4,1,1,8.0
3,5,4,3,2,14.0
4,19,18,15,13,NaN


In [13]:
def level1(df):
    # 원하는 배열 입력
    # *wntO 와 wntD의 합은 같아야함

    # 기존 데이터 shape 알기
    df_copy = df.copy()
    value = df_copy.shape[0]

    # 기존 데이터 맨 가장자리에 원하는 값 넣어줌 (성장률 구할것)

    df[value] = pd.Series(wntO) # 시리즈 변환
    df.loc[value] = pd.Series(wntD) # .loc 는 row

    # 성장률 구하기
    # 희망값/원래값
    df['Ei'] = df[value]/df[value-1] 
    df.loc['Fj'] = df.loc[value]/df.loc[value-1]
    
    df = df.fillna(0) # 결측치 값 0으로 변경 (nan 값 있으면 sum 같은 함수 적용 안됨)
    apple = []

    # 유출통행량 보정값
    
    for i in range(len(df)-3):
        value2 = df.iloc[i,value-1]/sum(df.loc[i] * df.loc['Fj'])
        apple.append(value2)

    df['Li'] = pd.Series(apple)
    df = df.fillna(0)

    banana = []

    # 유입통행량 보정값

    for i in range(len(df)-3):
        value3 = df.loc[value-1,i]/sum(df[i]*df['Ei'])
        banana.append(value3)

    df.loc['Lj'] = pd.Series(banana)
    df = df.fillna(0)
    
    return df, df_copy

In [14]:
def level2(df, df_copy):   
    result = df_copy.applymap(lambda x: None)
    value = df_copy.shape[0]
    
    i = ''
    j = ''

    for i in range(value-1):
        for j in range(value-1):
            #print(i, ',', j)
            result.loc[i,j] = df.loc[i,j]*df.loc[i,'Ei']*df.loc['Fj'][j]*(df.loc[i,'Li'] + df.loc['Lj'][j])/2

    result = result.fillna(0)
    #result = result.round(0) # 반올림
    result

    for num in range(value-1):
        result.loc[num,value-1] = sum(result.loc[num]) # 가로 합

    for num in range(value-1):
        result.loc[value-1,num] = sum(result[num]) # 세로 합

    return result

In [15]:
df_result, df_copy = level1(df)
result = level2(df_result, df_copy)
result

cycle1,df_copy = level1(result)
cycle1 = level2(cycle1,df_copy)
cycle1

wntcyc = 10 # << 수정하셈

for s in range(2,wntcyc):
    #print(s)
    globals()['cycle{}'.format(s)] = level1(globals()['cycle{}'.format(s-1)])[0]
    globals()['cycle{}'.format(s)] = level2(globals()['cycle{}'.format(s)], df_copy)
    globals()['cycle{}'.format(s)]

### 성장율이 1이 될때까지 돌림

In [16]:
cycle9

,0,1,2,3,4
0,7.076204,5.397706,5.114635,10.411455,28.0
1,5.812486,4.139208,7.056620,2.991685,20.0
2,3.053791,5.437328,1.544623,1.964258,12.0
3,7.057518,5.025757,4.284122,3.632602,20.0
4,23.000000,20.000000,18.000000,19.000000,0.0


### 프라타 과정

In [6]:
cycle1

,0,1,2,3,4,5,Ei
0,7.105676,5.433806,5.134697,10.401610,28.075789,28.0,0.997301
1,5.811136,4.148644,7.053278,2.975769,19.988827,20.0,1.000559
2,3.041985,5.429916,1.538279,1.946706,11.956886,12.0,1.003606
3,7.052362,5.034703,4.279958,3.611475,19.978498,20.0,1.001076
4,23.011159,20.047069,18.006212,18.935560,0.000000,NaN,NaN
5,23.000000,20.000000,18.000000,19.000000,NaN,NaN,NaN
Fj,0.999515,0.997652,0.999655,1.003403,NaN,NaN,NaN


In [7]:
cycle2

,0,1,2,3,4,5,Ei
0,7.080457,5.403033,5.117448,10.409873,28.010812,28.0,0.999614
1,5.811941,4.140415,7.055598,2.989153,19.997107,20.0,1.000145
2,3.052337,5.436835,1.543809,1.961845,11.994826,12.0,1.000431
3,7.056908,5.027260,4.283533,3.629554,19.997255,20.0,1.000137
4,23.001643,20.007544,18.000388,18.990424,0.000000,NaN,NaN
5,23.000000,20.000000,18.000000,19.000000,NaN,NaN,NaN
Fj,0.999929,0.999623,0.999978,1.000504,NaN,NaN,NaN


In [8]:
cycle3

,0,1,2,3,4,5,Ei
0,7.076830,5.398452,5.115083,10.411234,28.001600,28.0,0.999943
1,5.812407,4.139357,7.056518,2.991316,19.999597,20.0,1.000020
2,3.053577,5.437217,1.544513,1.963904,11.999210,12.0,1.000066
3,7.057429,5.025943,4.284064,3.632157,19.999593,20.0,1.000020
4,23.000243,20.000969,18.000178,18.998611,0.000000,NaN,NaN
5,23.000000,20.000000,18.000000,19.000000,NaN,NaN,NaN
Fj,0.999989,0.999952,0.999990,1.000073,NaN,NaN,NaN


In [9]:
cycle4

,0,1,2,3,4,5,Ei
0,7.076295,5.397815,5.114701,10.411421,28.000232,28.0,0.999992
1,5.812472,4.139227,7.056602,2.991629,19.999931,20.0,1.000003
2,3.053763,5.437317,1.544608,1.964208,11.999896,12.0,1.000009
3,7.057506,5.025784,4.284114,3.632537,19.999941,20.0,1.000003
4,23.000036,20.000143,18.000026,18.999795,0.000000,NaN,NaN
5,23.000000,20.000000,18.000000,19.000000,NaN,NaN,NaN
Fj,0.999998,0.999993,0.999999,1.000011,NaN,NaN,NaN


In [10]:
cycle5

,0,1,2,3,4,5,Ei
0,7.076217,5.397722,5.114645,10.411450,28.000034,28.0,0.999999
1,5.812484,4.139210,7.056618,2.991677,19.999990,20.0,1.000001
2,3.053787,5.437327,1.544621,1.964250,11.999985,12.0,1.000001
3,7.057517,5.025761,4.284121,3.632592,19.999991,20.0,1.000000
4,23.000005,20.000019,18.000005,18.999970,0.000000,NaN,NaN
5,23.000000,20.000000,18.000000,19.000000,NaN,NaN,NaN
Fj,1.000000,0.999999,1.000000,1.000002,NaN,NaN,NaN


In [ ]:
##... 기타 생략